In [4]:
import numpy as np

def viterbi_algorithm(observations, states, start_prob, trans_prob, emit_prob):
    """
    Viterbi algorithm implementation with explicit formula correspondence
    """
    T = len(observations)  # Total number of observations
    V = [{}]  # Delta values δₜ(i)
    psi = [{}]  # Backpointer values ψₜ(i)
    path = {}

    # Initialization phase: δ₁(i) = πᵢbᵢ(o₁)
    for state in states:
        pi_i = start_prob[state]  # πᵢ
        b_i_o1 = emit_prob[state][observations[0]]  # bᵢ(o₁)
        V[0][state] = pi_i * b_i_o1  # δ₁(i)
        path[state] = [state]

    # Recursion phase: δₜ(j) = max[δₜ₋₁(i)aᵢⱼ] · bⱼ(oₜ)
    for t in range(1, T):
        V.append({})
        psi.append({})
        newpath = {}

        for curr_state in states:
            # Calculate δₜ(j) and ψₜ(j)
            max_prob, best_prev_state = max(
                (V[t-1][prev_state]  # δₜ₋₁(i)
                 * trans_prob[prev_state][curr_state]  # aᵢⱼ
                 * emit_prob[curr_state][observations[t]],  # bⱼ(oₜ)
                 prev_state)
                for prev_state in states
            )

            V[t][curr_state] = max_prob  # Store δₜ(j)
            psi[t][curr_state] = best_prev_state  # Store ψₜ(j)
            newpath[curr_state] = path[best_prev_state] + [curr_state]

        path = newpath

    # Termination phase: P* = max[δT(i)] and q*T = Argmax[δT(i)]
    prob_star, q_star_T = max(
        (V[T-1][state], state)
        for state in states
    )

    return path[q_star_T], prob_star  # Return best path and its probability

# Define the model parameters
states = ['D', 'M', 'E']  # Difficult, Medium, Easy
observations = ['FB', 'FB', 'S', 'B', 'B', 'S', 'B', 'B', 'NS', 'B', 'B', 'S']

# Starting probabilities (assuming equal probability for each state)
start_prob = {
    'D': 1/3,
    'M': 1/3,
    'E': 1/3
}

# Transition probabilities
trans_prob = {
    'D': {'D': 1/3, 'M': 1/3, 'E': 1/3},
    'M': {'D': 0.5, 'M': 0.25, 'E': 0.25},
    'E': {'D': 0.5, 'M': 0.25, 'E': 0.25}
}

# Emission probabilities
emit_prob = {
    'D': {'FB': 0.1, 'B': 0.2, 'S': 0.4, 'NS': 0.3},
    'M': {'FB': 0.15, 'B': 0.25, 'S': 0.5, 'NS': 0.1},
    'E': {'FB': 0.2, 'B': 0.3, 'S': 0.4, 'NS': 0.1}
}

# Run the Viterbi algorithm
most_likely_sequence, probability = viterbi_algorithm(observations, states, start_prob, trans_prob, emit_prob)

print("Most likely sequence of test difficulties:", ' '.join(most_likely_sequence))
print("Probability of this sequence:", probability)

Most likely sequence of test difficulties: E E D E D M D E D E D M
Probability of this sequence: 2.7777777777777777e-12
